In [71]:
import requests
import aiohttp
import asyncio
import json
from sklearn.datasets import make_classification
import time 

BASE_URL = "http://127.0.0.1:8000"

In [72]:
def generate_heavy_dataset(n_samples=15000, n_features=50, n_informative=30):
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_redundant=10,
        n_classes=2,
        random_state=42
    )
    print("Генерация завершена.")
    return X.tolist(), y.tolist()

def check_response(response):
    """Проверяет HTTP-ответ и выводит результат."""
    if response.status_code == 200:
        print("   Ответ сервера:", response.json())
    else:
        print(f"Ошибка: (Статус-код: {response.status_code})")
        try:
            print("   Детали ошибки:", response.json())
        except json.JSONDecodeError:
            print("   Не удалось декодировать JSON из ответа.")

X_heavy, y_heavy = generate_heavy_dataset()

Генерация завершена.


In [80]:
model_configs = [
    {
        "config": {
            "model_name": "lgbm",
            "model_type": "LGBMClassifier",
            "n_estimators": 200, 
            "num_leaves": 40
        }
    },
    {
        "config": {
            "model_name": "rf",
            "model_type": "RandomForestClassifier",
            "n_estimators": 150, 
            "max_depth": 15
        }
    }
]

__Асихронная отправка запросов на обучение с использованием `aiohttp`__.
__Пример с эндпоинтом `/fit`__

In [107]:
print("--- Запуск последовательного (синхронного) обучения ---")
start_time_sequential = time.monotonic()

for config in model_configs:
    model_name = config['config']['model_name']
    print(f"-> Отправка запроса для модели: '{model_name}'")
    
    # 1. Собираем payload для текущей модели
    payload = {
        "X": X_heavy,
        "y": y_heavy,
        **config
    }
    response = requests.post(f"{BASE_URL}/fit", json=payload)
    print(response.json())
    end_time_sequential = time.monotonic()
print(f"\n--- Последовательное обучение завершено за {end_time_sequential - start_time_sequential:.4f} секунд ---")


--- Запуск последовательного (синхронного) обучения ---
-> Отправка запроса для модели: 'lgbm'
{'detail': "Модель по пути 'saved_models/lgbm.joblib' уже существует."}
-> Отправка запроса для модели: 'rf'
{'detail': "Модель по пути 'saved_models/rf.joblib' уже существует."}

--- Последовательное обучение завершено за 2.5685 секунд ---


In [105]:
response = requests.post(f"{BASE_URL}/unload", json={"model_name": "rf"})
check_response(response)
response = requests.post(f"{BASE_URL}/unload", json={"model_name": "lgbm"})
check_response(response)
response = requests.post(f"{BASE_URL}/remove_all")
check_response(response )

   Ответ сервера: {'message': "Модель 'rf' уже выгружена из памяти."}
   Ответ сервера: {'message': "Модель 'lgbm' уже выгружена из памяти."}
   Ответ сервера: {'message': "Все модели из директории 'saved_models' были успешно удалены."}


In [106]:
async def fit_model_async(session, payload):
    """Асинхронная функция для отправки запроса на обучение."""
    model_name = payload['config']['model_name']
    print(f"Отправка асинхронного запроса на обучение '{model_name}'...")
    
    async with session.post(f"{BASE_URL}/fit", json=payload) as response:
        if response.status == 200:
            print(f"Сервер принял задачу на обучение '{model_name}'")
            print("Ответ:", await response.йjson())
        else:
            print(f"Ошибка при отправке задачи на обучение '{model_name}' (Статус: {response.status})")
            print("   Детали:", await response.json())

async def main_fit(model_configs):
    start_time_async = time.monotonic()
    
    async with aiohttp.ClientSession() as session:
        tasks = []
        for config in model_configs:
            payload = {
                "X": X_heavy,
                "y": y_heavy,
                **config
            }
            task = asyncio.create_task(fit_model_async(session, payload))
            tasks.append(task)
        
        await asyncio.gather(*tasks)

    end_time_async = time.monotonic()

start = time.monotonic()
await main_fit(model_configs)
end_time_sequential = time.monotonic()
print(f"\n--- Последовательное обучение завершено за {end_time_sequential - start_time_sequential:.4f} секунд ---")


Отправка асинхронного запроса на обучение 'lgbm'...
Отправка асинхронного запроса на обучение 'rf'...
Ошибка при отправке задачи на обучение 'lgbm' (Статус: 400)
   Детали: {'detail': 'Модель lgbm успешно загружена и находится в saved_models/lgbm.joblib'}
Ошибка при отправке задачи на обучение 'rf' (Статус: 400)
   Детали: {'detail': 'Модель rf успешно загружена и находится в saved_models/rf.joblib'}

--- Последовательное обучение завершено за 67.7363 секунд ---


# __Пример с эндпоинтом `/load`__

In [82]:
models_to_load = ["lgbm", "rf"]
for model_name in models_to_load:
    response = requests.post(f"{BASE_URL}/load", json={"model_name": model_name})
    check_response(response )

   Ответ сервера: {'message': "Модель 'lgbm' успешно загружена в память."}
Ошибка: (Статус-код: 409)
   Детали ошибки: {'detail': "Достигнут лимит одновременно загруженных моделей (1). Текущие загруженные модели: ['lgbm']. Выгрузите одну из них перед загрузкой новой."}


__Пример с эндпоинтом `/predict`__

In [83]:
model_name = 'lgbm'

predict_config = {
    'X': X_heavy,
    'config': {
        'model_name': model_name
    }
}

response = requests.post(f"{BASE_URL}/predict", json=predict_config)
pred = response.json()['predictions']
print(pred[:100])

[1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0]


__Пример с эндпоинтом `/unload`__

In [89]:
response = requests.post(f"{BASE_URL}/unload", json={"model_name": "rf"})
check_response(response)
response = requests.post(f"{BASE_URL}/unload", json={"model_name": "lgbm"})
check_response(response)

   Ответ сервера: {'message': "Модель 'rf' уже выгружена из памяти."}
   Ответ сервера: {'message': "Модель 'lgbm' уже выгружена из памяти."}


__Пример с эндпоинтом `/remove`__

In [90]:
response = requests.post(f"{BASE_URL}/remove", json={"model_name": "lgbm"})
check_response(response)

   Ответ сервера: {'message': "Файл модели 'lgbm' успешно удален с диска."}


__Пример с эндпоинтом `/remove_all`__

In [91]:
response = requests.post(f"{BASE_URL}/remove_all")
check_response(response )

   Ответ сервера: {'message': "Все модели из директории 'saved_models' были успешно удалены."}
